#W207 Machine Learning Final Project #
###Forest Cover Type Prediction###
Amitava Das & Katherine Shelley

###Data Dictionary###

In [6]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, hstack

#plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

#libraries for cleaning/feature selection
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion

#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GMM

#metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import euclidean_distances

##Part 1: Data Collection and Baseline Estimation##
First we will download the data and split it into a training, development, and test sets.

###Load the data###
First let's load the data into a Pandas dataframe. The data has a lot of dummy binary features, which means there will be a lot of entries that are zero and we'll be adding more binary features to the data set. We will use a sparse representation in both Pandas and Numpy to save space and reduce computation time. Read more about Pandas sparse representation here: http://pandas.pydata.org/pandas-docs/stable/sparse.html

In [7]:
#read the training data into a pandas dataframe
data = os.path.join(os.getcwd(),'train.csv')
df_train = pd.read_csv(data).to_sparse() 
#separate out the training labels, drop the Id column from the training dataset
Y = np.array(df_train['Cover_Type']) #extract the labels
df_train = df_train.drop(['Id','Cover_Type'],1)
X = df_train.as_matrix()

#read the test data into a pandas frame
data = os.path.join(os.getcwd(),'test.csv')
df_test = pd.read_csv(data).to_sparse()
#separate out the Id column, keep it for Kaggle submissions
test_ids = np.array(df_test['Id'])
df_test = df_test.drop(['Id'],1)
test_data = df_test.as_matrix()

#You can see we have loaded the data into sparse dataframes
print type(df_train)
print df_train.shape
print type(df_test)
print df_test.shape

<class 'pandas.sparse.frame.SparseDataFrame'>
(15120, 54)
<class 'pandas.sparse.frame.SparseDataFrame'>
(565892, 54)


###Baseline model###
We initially created a baseline model using the K-Nearest Neighbors classifier with n_neighbors set to 1. With this baseline we were able to obtain an accuracy of 71.06% against the test set.

In [4]:
def InitialBaseline(X,Y,test_data,test_ids):
    # Instantiate the KNeighborsClassifier class with appropriate k for entire data set
    clf = KNeighborsClassifier(n_neighbors=1)
    clf.fit(X,Y)
    predicted = clf.predict(test_data)
    outfile = zip(test_ids, predicted)
    np.savetxt("BaseSubmission.csv", outfile, fmt='%i', delimiter=',', newline='\n', header='Id, Cover_Type',comments='')
    print "Saved ", len(outfile),"records to BaseSubmission.csv"

InitialBaseline(X,Y,test_data,test_ids)

Saved  565892 records to BaseSubmission.csv


##Part 2: Exploring Different Models##
Next we will use Grid Search to analyze various models to see if we get improvement simply by switching from Nearest Neighbors.

In [3]:
# Shuffle the inputs X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
X, Y = shuffle(X,Y,random_state=0) #shuffle the sparse matrix and dense array

# Set some variables to hold dev and training data;
N = X.shape[0]
#test_data, test_labels = X[2*N/3:], Y[2*N/3:]
dev_data, dev_labels = X[2*N/3:], Y[2*N/3:]
train_data, train_labels = X[:2*N/3], Y[:2*N/3]

print "Training data: %s" % str(train_data.shape)
print "Development data: %s" % str(dev_data.shape)


Training data: (10080, 54)
Development data: (5040, 54)


###Gaussian Mixture Model###
Perhaps a clustering model will give us better results. Since we know we have 7 cover types we will test various covariance types for a 7 component mixture model.

In [8]:

def components(p,g,type_str):
    #returns number of GMM components given features, gaussians, covariance type
    if type_str == 'full': 
        return (2*p + p*(p-1)/2)*g
    elif type_str == 'diag': 
        return (2*p)*g
    elif type_str == 'spherical': 
        return (p+1)*g
    elif type_str == 'tied': 
        return p*g + (p+ p*(p-1)/2)
    else: 
        return 9999
    
def get_n_components(n,classes=2):
    #returns a list of tuples representing PCA/GMM parameters
    #with less than n components
    params = []
    for i in range(1,n+1):
        for j in range(1,n+1):
            for cov_type in ['spherical','diag','tied','full']:
                #if the total components is less than or equal to 50
                if classes*components(i,j,cov_type) <= n:
                    params.append((i,j,cov_type))
    return params

def GMM_test():
    #Different component sizes for different covariance matrices:
    #Full: n means, n variances, and n(n-1)/2 covariances (symmetric)  per Gaussian
    #Tied: n means per Gaussian, n variances, and n(n-1)/2 covariances for all Gaussians 
    #Spherical: n means, 1 variance = n+1 per Gaussian
    #Diagonal: n means, n variances = 2*n per Gaussian
    best_pca,best_components,best_cov,best_accuracy = 0,0,'',0 #initialize the best parameters
    for i,j,cov_type in get_n_components(50,7):
        #initiate the PCA model, fit to training data, transform both train and dev data
        pca = PCA(n_components = i)
        reduced_train = pca.fit_transform(train_data)
        reduced_dev = pca.transform(dev_data)
        #split out the training examples
        train1 = reduced_train[np.where(train_labels==1)[0]] #type 1
        train2 = reduced_train[np.where(train_labels==2)[0]] #type 2
        train3 = reduced_train[np.where(train_labels==3)[0]] #type 3
        train4 = reduced_train[np.where(train_labels==4)[0]] #type 4
        train5 = reduced_train[np.where(train_labels==5)[0]] #type 5
        train6 = reduced_train[np.where(train_labels==6)[0]] #type 6
        train7 = reduced_train[np.where(train_labels==7)[0]] #type 7

        #initiate and fit a GMM for the type 1 reduced data
        gmm1 = GMM(n_components=j,covariance_type=cov_type)
        gmm1.fit(train1)
        pred1 = np.exp(gmm1.score(reduced_dev)) #predict the probability of positive for the reduced test data
        
        #initiate and fit a GMM for the type 2 reduced data
        gmm2 = GMM(n_components=j,covariance_type=cov_type)
        gmm2.fit(train2)
        pred2 = np.exp(gmm2.score(reduced_dev)) #predict the probability of positive for the reduced test data
        
        #initiate and fit a GMM for the type 3 reduced data
        gmm3 = GMM(n_components=j,covariance_type=cov_type)
        gmm3.fit(train3)
        pred3 = np.exp(gmm3.score(reduced_dev)) #predict the probability of positive for the reduced test data
        
        #initiate and fit a GMM for the type 4 reduced data
        gmm4 = GMM(n_components=j,covariance_type=cov_type)
        gmm4.fit(train4)
        pred4 = np.exp(gmm4.score(reduced_dev)) #predict the probability of positive for the reduced test data
        
        #initiate and fit a GMM for the type 5 reduced data
        gmm5 = GMM(n_components=j,covariance_type=cov_type)
        gmm5.fit(train5)
        pred5 = np.exp(gmm5.score(reduced_dev)) #predict the probability of positive for the reduced test data
        
        #initiate and fit a GMM for the type 6 reduced data
        gmm6 = GMM(n_components=j,covariance_type=cov_type)
        gmm6.fit(train6)
        pred6 = np.exp(gmm6.score(reduced_dev)) #predict the probability of positive for the reduced test data
        
        #initiate and fit a GMM for the type 7 reduced data
        gmm7 = GMM(n_components=j,covariance_type=cov_type)
        gmm7.fit(train7)
        pred7 = np.exp(gmm7.score(reduced_dev)) #predict the probability of negative for the reduced test data
        
        probs = np.max(np.array([pred1,pred2,pred3,pred4,pred5,pred6,pred7]),axis=0) #get the max probabilities
        
        prediction = np.zeros(dev_labels.shape) #set a vector of zeros for predictions
        
        prediction[np.where(pred1 == probs)[0]]=1 #predicted 1
        prediction[np.where(pred2 == probs)[0]]=2 #predicted 2
        prediction[np.where(pred3 == probs)[0]]=3 #predicted 3
        prediction[np.where(pred4 == probs)[0]]=4 #predicted 4
        prediction[np.where(pred5 == probs)[0]]=5 #predicted 5
        prediction[np.where(pred6 == probs)[0]]=6 #predicted 6
        prediction[np.where(pred7 == probs)[0]]=7 #predicted 7

        accuracy = np.mean(prediction==dev_labels)
        if accuracy > best_accuracy:
            best_pca = i
            best_components = j
            best_cov = cov_type
            best_accuracy = accuracy
    print "Greatest Accuracy: %s Using %s PCA components, %s Gaussians, with %s Covariance Type." % (best_accuracy,best_pca,best_components,best_cov)
GMM_test()

Greatest Accuracy: 0.508333333333 Using 3 PCA components, 1 Gaussians, with diag Covariance Type.


###Logistic Regression###
Gaussian Mixture Models did not give us very good accuracy. Let's see if we can do better with Logistic Regression. Logistic Regression is a popular choice for classification because 

In [10]:
lm = LogisticRegression() #initialize a logistic regression model
parameters = {'C':[10**i for i in range(-5,5)],'penalty':['l1','l2']}
clf = GridSearchCV(lm, parameters)
clf.fit(train_data,train_labels)
print '%s Accuracy using C = %s and Penalty Function %s' % (clf.score(dev_data,dev_labels)*100,clf.best_params_['C'],clf.best_params_['penalty'])

67.6388888889 Accuracy using C = 10000 and Penalty Function l1


###Random Forests###
Now Let's test Random Forests for various values of n trees. Note, this cell can take several minutes to run.

In [5]:
parameters = {'n_estimators':[10**i for i in range(4)],'criterion':['gini','entropy']}

rf = RandomForestClassifier()
clf = GridSearchCV(rf, parameters)
clf.fit(train_data,train_labels)
print 'Random Forest gives %s percent Accuracy using n = %s trees and %s criterion' % (clf.score(dev_data,dev_labels)*100,clf.best_params_['n_estimators'],clf.best_params_['criterion'])


Random Forest gives 86.0714285714 percent Accuracy using n = 1000 trees and gini criterion


Random forests have given by far the best accuracy that we have seen. This is not surprising considering the amount of binary features in our data set. From here on we will use this model.

##Part 3: Improving Our Model##
Next we will explore feature engineering and dimensionality reduction to try to improve the accuracy of our Random Forest classifier.

###Feature Engineering###
Next we will perform feature engineering:
1. Create new variables that represent the climatic and geologic soil zones (soil id prefix values)
2. Convert degrees fields to radians.
3. Add new binary variables to represent degree field "quadrants", i.e. 0 - 90 degrees, 90 to 180 degrees, etc. (in radians)
4. Normalize hillshade features from 0 to 255 to 0 to 1 


In [8]:
#Functions for feature engineering
def add_soil_prefix_fields(df):
    #adds new variables to a dataframe that represent the climatic and geologic zones 
    #(first two digits of the soil type variable)
    #we'll use a dicitonary to isolate the soil id's by their climatic/geologic prefix
    soil_id = {'27':['1','2','3','4','5','6'],
               '35':['7','8'],
               '42':['9'],
               '47':['10','11','12','13'],
               '51':['14','15'],
               '61':['16','17'],
               '67':['18'],
               '71':['19','20','21'],
               '72':['22','23'],
               '77':['24','25','26','27','28','29','30','31','32','33','34'],
               '87':['35','36','37','38','39','40']
              }

    climatic = {'2':['1','2','3','4','5','6'],
                '3':['7','8'],
               '4':['9','10','11','12','13'],
               '5':['14','15'],
               '6':['16','17','18'],
               '7':['19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34'],
               '8':['35','36','37','38','39','40']
               }

    geologic = {'1':['14','15','16','17','19','20','21'],
                '2':['9','22','23'],
                '5':['7','8'],
                '7':['1','2','3','4','5','6','10','11','12','13','18','24','25','26','27','28',
                     '29','30','31','32','33','34','35','36','37','38','39','40']
               }

    #add new variables for the soil type prefix, climatic zone, and geologic zone
    for key in soil_id:
        clim,geol = key[0],key[1]
        df_clim = df[['Soil_Type%s' % i for i in climatic[clim]]] #subset of df of climatic types
        df_geol = df[['Soil_Type%s' % i for i in geologic[geol]]] #subset of df of climatic types

        df['climatic zone %s' % clim] = np.sum(np.asarray(df_clim),axis=1)
        df['geologic zone %s' % geol] = np.sum(np.asarray(df_geol),axis=1)
        del df_clim
        del df_geol
    return df

def degrees_to_radians(df):
    for col in ['Aspect','Slope']:
        df[col] = np.radians(df[col].as_matrix())
    return df

def add_aspect_quadrants(df):
    df['Aspect0_90'] = np.where(df['Aspect']<0.5,np.ones(df.shape[0]),np.zeros(df.shape[0]))
    df['Aspect90_180'] = np.where(df['Aspect']<1,np.ones(df.shape[0]),np.zeros(df.shape[0])) - np.array(df['Aspect0_90'])
    df['Aspect180_270'] = np.where(df['Aspect']<1.5,np.ones(df.shape[0]),np.zeros(df.shape[0])) - np.array(df['Aspect0_90'] + df['Aspect90_180'])
    df['Aspect270_360'] = np.where(df['Aspect']>=1.5,np.ones(df.shape[0]),np.zeros(df.shape[0]))
    return df

def scale_shade_value(df):
    for col in ['Hillshade_9am','Hillshade_Noon','Hillshade_3pm']:
        df[col] = df[col]*(1.0/255)
    return df

#add fields for the soil type climatic and geologic zone prefixes
df_train = scale_shade_value(add_aspect_quadrants(degrees_to_radians(add_soil_prefix_fields(df_train))))
df_test = scale_shade_value(add_aspect_quadrants(degrees_to_radians(add_soil_prefix_fields(df_test))))

print df_train.shape


(15120, 69)


###Sparse Matrices###
For our feature engineering we will convert the test dataframe to a SciPy sparse matrix. In particular we will use a Compressed Sparse Row matrix. Read more about Compressed Sparse Row matrices and their operations here: http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix

In [9]:
#variables to scale
real_valued = ['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways',
       'Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points']

X_real = df_train[real_valued].as_matrix()
df_train = df_train.drop(real_valued,1)
X = df_train.as_matrix()

test_real = df_test[real_valued].as_matrix()
df_test = df_test.drop(real_valued,1)
test_data = csr_matrix(df_test)

#scale and transform the (dense) real-valued features of the training and test data
scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_real)
scaler.transform(test_real)

X = np.hstack([X_real,X])
test_data = hstack([test_real,test_data])

#delete data frames and temporary data arrays
del df_train
del df_test
del X_real
del test_real

print "New training size: %s" % str(X.shape)
print type(X)
print "New test size: %s" % str(test_data.shape)
print type(test_data)


New training size: (15120, 69)
<type 'numpy.ndarray'>
New test size: (565892, 69)
<class 'scipy.sparse.coo.coo_matrix'>


####Scale the data, convert to Numpy arrays####

In [5]:
# Shuffle the inputs X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.

#First we will convert the training data set back to a dense array, the test data will always stay in a sparse matrix
try:
    X = X.toarray()
except:
    pass

X, Y = shuffle(X,Y,random_state=0) #shuffle the sparse matrix and dense array
print 'Training data shape: ', X.shape
print 'Tranining label shape: ', Y.shape
#print 'Test data shape: ', test_data.shape


Training data shape:  (15120, 69)
Tranining label shape:  (15120,)


###Improving the model###
Now that we have a baseline prediciton score, let's see what we can do to improve it. First we will separate our training data into training, development, and test sets. We will not use the test set until we have finalized our model. Tuning will be performed with the development set only to avoid overfitting to the test set.

In [6]:
# Set some variables to hold dev and training data;
N = X.shape[0]
#test_data, test_labels = X[2*N/3:], Y[2*N/3:]
dev_data, dev_labels = X[2*N/3:], Y[2*N/3:]
train_data, train_labels = X[:2*N/3], Y[:2*N/3]

print "Training data: %s" % str(train_data.shape)
print "Development data: %s" % str(dev_data.shape)

Training data: (10080, 69)
Development data: (5040, 69)


In [7]:
poly = preprocessing.PolynomialFeatures(degree=2)
poly.fit_transform(train_data)
poly.transform(dev_data)

rf = RandomForestClassifier(n_estimators=1000,criterion='gini')
rf.fit(train_data,train_labels)
print 'Random Forest gives %s percent Accuracy ' % (rf.score(dev_data,dev_labels)*100)

Random Forest gives 84.7222222222 percent Accuracy 


###Feature Selection###

In [8]:
#let's run PCA on the data but not set any component value so we can determine how many principal components we should keep 
#to capture as much of the variance as possible.
pca = PCA()
pca.fit(train_data)

var = 0 #retained variance
k = 0 #number of components
while var < 0.99:
    k+=1
    var = np.cumsum(pca.explained_variance_ratio_,axis=0)[k]
print "Optimal number of principal components is %s" % k

Optimal number of principal components is 3


In [16]:
# Let's reduce the dimensionality with PCA
pca = PCA(n_components=3)

# Though maybe some original features where good, let's keep some features in their original form
selection = SelectKBest(k=50)

# Build estimator from PCA and Univariate selection
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
train_data_red = combined_features.fit(train_data,train_labels).transform(train_data)
dev_data_red = combined_features.transform(dev_data)

'''
pipeline = Pipeline([("features", combined_features), ("rf", rf)])

param_grid = dict(features__pca__n_components=[3],
                  features__univ_select__k=[1,2,3,5,10,15,20,25,40,50])
clf = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
clf.fit(train_data, train_labels)

'''

rf = RandomForestClassifier(n_estimators=1000,criterion='gini')
rf.fit(train_data_red,train_labels)
print 'Random Forest gives %s percent Accuracy' % (rf.score(dev_data_red,dev_labels)*100)

Random Forest gives 87.9761904762 percent Accuracy


In [11]:
def BestModelBaseline():
    # Instantiate the KNeighborsClassifier class with appropriate k for entire data set
    # Let's reduce the dimensionality with PCA
    pca = PCA(n_components=3)

    # Though maybe some original features where good, let's keep some features in their original form
    selection = SelectKBest(k=50)

    # Build estimator from PCA and Univariate selection
    combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
    X_red = combined_features.fit_transform(X,Y)
    test_data_red = combined_features.transform(test_data)
    
    clf = RandomForestClassifier(n_estimators=1000)
    clf.fit(X_red,Y)
    predicted = clf.predict(test_data_red)
    outfile = zip(test_ids, predicted)
    np.savetxt("Submission3.csv", outfile, fmt='%i', delimiter=',', newline='\n', header='Id, Cover_Type',comments='')
    print "Saved ", len(outfile),"records to Submission2.csv"
BestModelBaseline()

C:\Python27\lib\site-packages\sklearn\feature_selection\univariate_selection.py:111: UserWarning: Features [16 47] are constant.
  UserWarning)


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.